In [1]:
from IPython.display import clear_output

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException

import pandas as pd
import numpy as np

import json
import time
import mouse
import os
from os.path import exists


## Парсим категории верхних уровней

In [2]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)

In [11]:
# Открываем страницу перека
url = 'https://www.perekrestok.ru/cat'
driver.get(url)

# Сохраняем html с верхними уровнями категорий перектрестка
with open("perek_cats.html", "w", encoding='utf-8') as file:
    file.write(driver.page_source)

In [20]:
global_elements = soup.find_all('li', class_='catalog-menu__first-category-item')
global_elements[1]

<li class="catalog-menu__first-category-item"><a class="sc-jLiVFv fMXfbe" href="/cat/mc/1103/v-novyj-god-s-polzoj">В Новый год с пользой<svg color="none" fill="none" height="8" viewbox="0 0 16 17" width="8" xmlns="http://www.w3.org/2000/svg"><path d="M4.96 15.17l6.81-6.67-6.8-6.67" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="3"></path></svg></a></li>

In [12]:
# Парсим нижние уровни категорий с html страницы 
with open('perek_cats.html', encoding='utf-8') as file:
    txt = file.read()

soup = BeautifulSoup(txt, 'lxml')
global_elements = soup.find_all('div', class_='category-filter-item__header')

main_table = pd.DataFrame(columns = ['level_0', 'level_1', 'level_1_href'])

for global_element in global_elements:
    # Наименование первого уровня
    level_0_name = global_element.find('a', class_='sc-dQoVA dsEnQi sc-fTACoA bOLdeS sc-bTRMho cDAjZl').text
    # Наименования второго уровня
    level_1_names = global_element.find_all('a', class_='sc-dQoVA dsEnQi sc-fTACoA bOLdeS sc-fTNIjK hGWjxo')
    level_1_names = [name.text for name in level_1_names]
    # Ссылки на элементы второго уровня
    level_1_hrefs = global_element.find_all('a', class_='sc-dQoVA dsEnQi sc-fTACoA bOLdeS sc-fTNIjK hGWjxo')
    level_1_hrefs = [url.rstrip('/cat') + name.get('href') for name in level_1_hrefs]
    append_table = pd.DataFrame({'level_0': [level_0_name]*len(level_1_names), 
                                 'level_1': level_1_names, 
                                 'level_1_href': level_1_hrefs})
    main_table = main_table.append(append_table, ignore_index=True)
    
main_table.to_excel('./data/perek_categories.xlsx', index=False)

In [13]:
main_table

,level_0,level_1,level_1_href


## Парсинг айтем неймов

In [9]:
# Подгружаем спарсенный категоризатор
main_table = pd.read_excel('./data/perek_categories.xlsx')
url_list = main_table['level_1_href'].tolist()

In [3]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

The chromedriver version (115.0.5790.170) detected in PATH at C:\Users\lokri\Jupyter_projects\Parsing\chromedriver.exe might not be compatible with the detected chrome version (116.0.5845.187); currently, chromedriver 116.0.5845.96 is recommended for chrome 116.*, so it is advised to delete the driver in PATH and retry


In [6]:
# Настраиваем костыльный счетчик
if os.path.exists('counter_perek.txt'):
    with open('counter_perek.txt') as f:
        i = int(f.readline())
else:
    i = 0
i

0

In [10]:
# Парсим айтем неймы из перека
file_name = './data/perek_item_sample.csv'

# Обработка подтверждения возраста
def click_adult_button():
    '''
    Нажатие кнопки проверки возраста в Переке
    '''
    try:
        adult_control = driver.find_elements(By.CLASS_NAME, value='adult-modal-controls')
        if adult_control:
            adult_control[0].find_element(By.TAG_NAME, value='button').click()
    except ElementNotInteractableException:
        pass
    
# Парсинг айтемнеймов
for url in url_list[i:]:
    driver.get(url)
    click_adult_button()
    # Поиск мелких категорий
    small_categories = driver.find_elements(By.CLASS_NAME, 'catalog-content-group__title')
    small_categories = [category.find_element(By.XPATH, '..') for category in small_categories]
    time.sleep(1)
    # Сбор данных по каждой из мелких категорий
    for category in small_categories:
        while True:
            try:
                # Наименование последнего уровня
                level_2_name = category.find_element(By.CLASS_NAME, 'catalog-content-group__title').text
                # Наименование продукта
                item_names = category.find_elements(By.CLASS_NAME, 'product-card__title')
                item_names = [item_name.text for item_name in item_names]
                # Цена
                prices = category.find_elements(By.CLASS_NAME, 'price-new')
                prices = [price.text.rstrip(' ₽').replace(',', '.') for price in prices]
            except StaleElementReferenceException:
                time.sleep(1)
                continue
            
            if all([level_2_name, item_names[0], prices[0]]):
                break
            else:
                time.sleep(0.1)
        
        pd.DataFrame(data = {'level_1_href': [url]*len(prices),
                             'level_2': [level_2_name]*len(prices),
                             'item_name': item_names,
                             'value': prices}).to_csv(file_name, 
                                                         mode='a' if exists(file_name) else 'w', 
                                                         sep='\t', 
                                                         index=False,
                                                         header = not exists(file_name))
    
    i += 1
    with open('counter_perek.txt', 'w') as f:
        f.write(str(i))
    time.sleep(1)
else:
    os.remove('counter_perek.txt')
        

KeyboardInterrupt: 